# Begin scrape_mars.py

In [149]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
import pandas as pd

In [6]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
# Define database and collection
db = client.mars_db
collection = db.articles

# Nasa Mars News

In [255]:
final_scrape_output ={}

In [42]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [235]:
# Retrieve the parent divs for all articles
news_title = soup.find('div', class_='content_title').text.strip()

In [256]:
final_scrape_output.update(
    {'news_title': news_title}
)

In [236]:
print(news_title)

NASA's Mars 2020 Rover Closer to Getting Its Name


In [239]:
# Retrieve news paragraph
news_p = soup.find('div', class_='rollover_description_inner').text.strip()

In [241]:
print(news_p)

155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.


# JPL Mars Space Images - Featured Image

In [246]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [247]:
# URL of page to be scraped
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url_img)

In [248]:
try:
    browser.select('category', 'featured')
    bg_attribute_string =browser.find_by_css('article').first['style']
    bg_attribute_string_left = bg_attribute_string.split('("',1)[1]
    bg_attribute_string_final = bg_attribute_string_left.split('")',1)[0]
    
except ElementDoesNotExist:
    print("Clicking Complete")

In [249]:
print(bg_attribute_string_final)

/spaceimages/images/wallpaper/PIA16726-1920x1200.jpg


In [251]:
featured_image_url = 'https://www.jpl.nasa.gov' + bg_attribute_string_final

In [252]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16726-1920x1200.jpg


# Mars Weather

In [131]:
# URL of page to be scraped
url_weather = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response_weather = requests.get(url_weather)

# Create BeautifulSoup object; parse with 'lxml'
soup_weather = BeautifulSoup(response_weather.text, 'lxml')

In [143]:
#Extract text from tweet
weather_p = soup_weather.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text.strip()

In [145]:
#Clean up text from tweet
mars_weather = weather_p.replace('\n',', ')

'InSight sol 421 (2020-02-02) low -93.1ºC (-135.6ºF) high -13.8ºC (7.2ºF), winds from the SSE at 5.6 m/s (12.5 mph) gusting to 21.0 m/s (46.9 mph), pressure at 6.30 hPapic.twitter.com/zgieOlktxs'

In [254]:
print(mars_weather)

InSight sol 421 (2020-02-02) low -93.1ºC (-135.6ºF) high -13.8ºC (7.2ºF), winds from the SSE at 5.6 m/s (12.5 mph) gusting to 21.0 m/s (46.9 mph), pressure at 6.30 hPapic.twitter.com/zgieOlktxs


# Mars Facts

In [147]:
url_facts = 'https://space-facts.com/mars/'

In [150]:
# Use Panda's `read_html` to parse the url
tables = pd.read_html(url_facts)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [165]:
#Load data into DataFrame
facts_df = tables[0]
facts_df.rename(columns={
    0:"Description",
    1:"Value"
}, inplace=True)
facts_df.set_index('Description',inplace=True)
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [167]:
#convert pandas into html table
facts_df.to_html('mars_facts_table.html')

In [260]:
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [169]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [179]:
# URL of page to be scraped
url_mars_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_mars_hemispheres)

In [233]:
#loop to get all mars hemispheres
hemisphere_image_urls = []

mars_hemishpheres = [
    "Valles Marineris Hemisphere",
    "Cerberus Hemisphere",
    "Schiaparelli Hemisphere",
    "Syrtis Major Hemisphere"
]

for hemisphere in mars_hemishpheres:
    hemisphere_image_urls_dict = {}
    browser.find_by_text(f'{hemisphere} Enhanced').click()
    img_url_href = browser.find_by_text('Sample')['href']
    hemisphere_image_urls_dict.update(
        {'title' : hemisphere,
        'img_url': img_url_href
        }
    )
    hemisphere_image_urls.append(hemisphere_image_urls_dict)
    browser.back()

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}]

In [ ]:
print(hemisphere_image_urls)

# END scrap_mars.py